Delta Class metric evaluation

In [2]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 100.0 if i == 3 else 0.001  # Introduce a bug in the third client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta class matrix
    delta_class_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            Δ_class, _, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_class_matrix[i, j] = Δ_class
            delta_class_matrix[j, i] = Δ_class  # Mirror the matrix
    
    # Print the delta class matrix
    print(f"Round {round_num} Delta Class Matrix:")
    print(delta_class_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(delta_class_matrix[:, np.newaxis] - delta_class_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    #print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84681666), ('loss', 0.5754327)]))])
Round 1 Delta Class Matrix:
[[   0. 1920. 1873. 8699. 2035. 1842. 1932. 2048. 2231. 1978.]
 [1920.    0. 1895. 8520. 1931. 1722. 1921. 2023. 2249. 1916.]
 [1873. 1895.    0. 8907. 1898. 1727. 1879. 1962. 2151. 1948.]
 [8699. 8520. 8907.    0. 8988. 8701. 8989. 8946. 8979. 8832.]
 [2035. 1931. 1898. 8988.    0. 1829. 1934. 1908. 2249. 1909.]
 [1842. 1722. 1727. 8701. 1829.    0. 1850. 1874. 2152. 1791.]
 [1932. 1921. 1879. 8989. 1934. 1850.    0. 1943. 2089. 1964.]
 [2048. 2023. 1962. 8946. 1908. 1874. 1943.    0. 2196. 2015.]
 [2231. 2249. 2151. 8979. 2249. 2152. 2089. 2196.    0. 2274.]
 [1978. 1916. 1948. 8832. 1909. 1791. 1964. 2015. 2274.    0.]]
Round 1 Distance Matrix:
[[    0.          2726.8030732   2666.39194418 23014.53716676
   2896.82153403  2639.5

delta score

In [3]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    Δ_class = np.sum(np.argmax(predictions_i, axis=1) != np.argmax(predictions_j, axis=1))
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((np.argmax(predictions_i, axis=1) == k) & (np.argmax(predictions_j, axis=1) == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the delta score matrix
    delta_score_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, Δ_score, _, _ = perform_differential_testing(predictions[i], predictions[j])
            delta_score_matrix[i, j] = Δ_score
            delta_score_matrix[j, i] = Δ_score  # Mirror the matrix
    
    # Print the delta score matrix
    print(f"Round {round_num} Delta Score Matrix:")
    print(delta_score_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(delta_score_matrix[:, np.newaxis] - delta_score_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    #print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84178334), ('loss', 0.5894773)]))])
Round 1 Delta Score Matrix:
[[    0. 98446. 98450. 98439. 98549. 98416. 98492. 98466. 98548. 98421.]
 [98446.     0. 41992. 33815. 46317. 35454. 38994. 42037. 41787. 34194.]
 [98450. 41992.     0. 39321. 50462. 40666. 43585. 45119. 46285. 39330.]
 [98439. 33815. 39321.     0. 42529. 30396. 34634. 38105. 37733. 28765.]
 [98549. 46317. 50462. 42529.     0. 44264. 45722. 48723. 48444. 42811.]
 [98416. 35454. 40666. 30396. 44264.     0. 36900. 39842. 39767. 31380.]
 [98492. 38994. 43585. 34634. 45722. 36900.     0. 42102. 41773. 35702.]
 [98466. 42037. 45119. 38105. 48723. 39842. 42102.     0. 45704. 38716.]
 [98548. 41787. 46285. 37733. 48444. 39767. 41773. 45704.     0. 38362.]
 [98421. 34194. 39330. 28765. 42811. 31380. 35702. 38716. 38362.     0.]]
Round 1 Dis

pks 

In [4]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    Δ_class = np.sum(np.argmax(predictions_i, axis=1) != np.argmax(predictions_j, axis=1))
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((np.argmax(predictions_i, axis=1) == k) & (np.argmax(predictions_j, axis=1) == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the P_KS matrix
    p_ks_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, p_ks, _ = perform_differential_testing(predictions[i], predictions[j])
            p_ks_matrix[i, j] = p_ks
            p_ks_matrix[j, i] = p_ks  # Mirror the matrix
    
    # Print the P_KS matrix
    print(f"Round {round_num} P_KS Matrix:")
    print(p_ks_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(p_ks_matrix[:, np.newaxis] - p_ks_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    #print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.8452), ('loss', 0.5765802)]))])
Round 1 P_KS Matrix:
[[0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
  0.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 1.83137819e-034 1.60237861e-033
  8.24767444e-034 1.09842603e-046 1.29910952e-135 8.23960467e-011
  1.16487224e-002 0.00000000e+000]
 [0.00000000e+000 1.83137819e-034 0.00000000e+000 9.02816892e-001
  5.31033198e-129 4.19277177e-003 4.66802256e-296 1.30235574e-007
  3.51353875e-050 1.53925232e-221]
 [0.00000000e+000 1.60237861e-033 9.02816892e-001 0.00000000e+000
  9.19210781e-118 7.17682488e-003 1.00836054e-278 9.30544470e-008
  1.86985806e-043 2.47407378e-237]
 [0.00000000e+000 8.24767444e-034 5.31033198e-129 9.19210781e-118
  0.00000000e+000 8

x2

In [5]:
import tensorflow_federated as tff
import tensorflow as tf
import numpy as np
from scipy.stats import ks_2samp, chi2_contingency
import nest_asyncio

nest_asyncio.apply()

# Set the local execution context
tff.backends.native.set_local_execution_context()

# Load and preprocess the MNIST dataset
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)))

mnist_train, mnist_test = tf.keras.datasets.mnist.load_data()
mnist_train = (mnist_train[0].reshape(-1, 28, 28), mnist_train[1])
mnist_test = (mnist_test[0].reshape(-1, 28, 28), mnist_test[1])

# Split the data into 10 clients
def create_client_data(data, labels, num_clients=10):
    client_data = []
    client_labels = []
    data_per_client = len(data) // num_clients
    for i in range(num_clients):
        client_data.append(data[i * data_per_client:(i + 1) * data_per_client])
        client_labels.append(labels[i * data_per_client:(i + 1) * data_per_client])
    return client_data, client_labels

client_data, client_labels = create_client_data(mnist_train[0], mnist_train[1])

# Define the metrics function
def perform_differential_testing(predictions_i, predictions_j):
    if predictions_i.ndim == 1:
        predictions_i = np.expand_dims(predictions_i, axis=1)
    if predictions_j.ndim == 1:
        predictions_j = np.expand_dims(predictions_j, axis=1)
    
    pred_class_i = np.argmax(predictions_i, axis=1)
    pred_class_j = np.argmax(predictions_j, axis=1)
    
    Δ_class = np.sum(pred_class_i != pred_class_j)
    Δ_score = np.sum(predictions_i != predictions_j)
    P_KS = ks_2samp(predictions_i.flatten(), predictions_j.flatten()).pvalue
    contingency = np.array([[np.sum((pred_class_i == k) & (pred_class_j == l)) for l in range(10)] for k in range(10)])
    contingency += 1  # Add-one smoothing
    P_X2 = chi2_contingency(contingency)[1]

    return Δ_class, Δ_score, P_KS, P_X2

# Create a simple model
def create_model():
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

# Create a federated learning process
def model_fn():
    model = create_model()
    return tff.learning.from_keras_model(
        model,
        input_spec=(tf.TensorSpec(shape=[None, 28, 28], dtype=tf.float32),
                    tf.TensorSpec(shape=[None], dtype=tf.int32)),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# Define the client optimizer function
def client_optimizer_fn():
    return tf.keras.optimizers.Nadam(learning_rate=0.001)

# Define the federated averaging process
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=client_optimizer_fn
)

# Initialize the process
state = iterative_process.initialize()

# Custom function to determine if a model is an outlier using a DBSCAN-like approach
def is_outlier(metric_data, epsilon=0.5, min_samples=2):
    num_points = metric_data.shape[0]
    distances = np.linalg.norm(metric_data[:, np.newaxis] - metric_data, axis=2)
    neighbors = np.sum(distances < epsilon, axis=1)
    outliers = neighbors < min_samples
    return outliers

# Standalone function for preprocessing
def preprocess_fn(x, y):
    return tf.cast(x, tf.float32) / 255.0, tf.cast(y, tf.int32)

# Simulate federated training
num_rounds = 10  # Define the number of rounds
num_clients = 10  # Define the number of clients

for round_num in range(1, num_rounds + 1):
    # Create TensorFlow datasets for each client
    federated_data = [
        tf.data.Dataset.from_tensor_slices((client_data[i], client_labels[i]))
        .map(preprocess_fn)
        .batch(20)
        for i in range(num_clients)
    ]
    
    # Perform a round of federated training
    state, metrics = iterative_process.next(state, federated_data)
    print(f'Round {round_num}, Metrics: {metrics}')
    
    # Get predictions for each client using MNIST test data
    predictions = []
    for i in range(num_clients):
        model = create_model()
        learning_rate = 10.0 if i == 0 else 0.001  # Introduce a bug in the first client
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        model.fit(client_data[i], client_labels[i], epochs=1, verbose=0)
        predictions.append(model.predict(mnist_test[0]))  # Use MNIST test data for predictions
    
    # Calculate the P_X2 matrix
    p_x2_matrix = np.zeros((num_clients, num_clients))
    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            _, _, _, p_x2 = perform_differential_testing(predictions[i], predictions[j])
            p_x2_matrix[i, j] = p_x2
            p_x2_matrix[j, i] = p_x2  # Mirror the matrix
    
    # Print the P_X2 matrix
    print(f"Round {round_num} P_X2 Matrix:")
    print(p_x2_matrix)
    
    # Calculate the distance matrix
    distance_matrix = np.linalg.norm(p_x2_matrix[:, np.newaxis] - p_x2_matrix, axis=2)
    
    # Print the distance matrix
    print(f"Round {round_num} Distance Matrix:")
    print(distance_matrix)
    
    # Calculate and print average distances
    average_distances = np.mean(distance_matrix, axis=1)
    print(f"Round {round_num} Average distances:")
    print(average_distances)
    
    # Detect outliers using the custom DBSCAN-like function
    outliers = is_outlier(distance_matrix)
    #print(f'Round {round_num}, Outliers: {outliers}')
    
    # Identify the client with the highest average distance
    max_distance_client = np.argmax(average_distances)
    print(f"Round {round_num}, Client with highest average distance: {max_distance_client}")


Round 1, Metrics: OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('value_sum_process', ()), ('weight_sum_process', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.84381664), ('loss', 0.5853298)]))])
Round 1 P_X2 Matrix:
[[0.         0.98368428 0.88147056 0.51300364 0.546374   0.84078703
  0.73490878 0.6248268  0.94027218 0.99131579]
 [0.98368428 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.88147056 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.51300364 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.546374   0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.84078703 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.73490878 0.         0.         0.         0.         0.
  0.         0.         0.         0.  